#### Part 2.3 Dynamic cartogram with voivodships (mobility in the area of ​​commerce and recreation) - displayed in a browser window

In [ ]:
# Import neccessary packages
!pip install geojson
import json
import pandas
import plotly.io as pio
pio.renderers.default = 'browser'

In [ ]:
# Re-import .csv files with mobility data
files = os.path.join('data/*PL_Mobility.csv')
files = glob.glob(files)
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df_en = df[['sub_region_1', 'iso_3166_2_code', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']]
df_en = df_en[df_en['iso_3166_2_code'].notna()]

# Import of a csv file with data on voivodship codes
path = 'data/codes.csv'
df_codes=pd.read_csv(path, sep = ';')
df_final=pd.merge(df_en, df_codes, how="left", left_on = "iso_3166_2_code", right_on = "KOD_ISO")

In [ ]:
# matching ID
poland_states = json.load(open("data/woj.geojson", "r"))
df_final.rename(columns = {"ID":"id"}, inplace = True)
df_final.rename(columns = {"Nazwa":"name"}, inplace = True)

state_id_map = {}
for feature in poland_states["features"]:
    feature["id"] = feature["properties"]["id"]
    print(feature["properties"]["id"])
    feature["properties"]["nazwa"] = df_final[df_final["id"]==feature["properties"]["id"]]["name"].unique()[0]
    print(feature["properties"]["nazwa"])

In [ ]:
# further data modifications, date format change
formated_gdf = df_final.fillna(0).groupby(['date', 'id', 'name'])['retail_and_recreation_percent_change_from_baseline'].mean()
formated_gdf = formated_gdf.reset_index()
formated_gdf['date'] = pd.to_datetime(formated_gdf['date'])
formated_gdf['date'] = formated_gdf['date'].dt.strftime('%m/%d/%Y')

In [ ]:
# modification of the date to calculate the monthly average
# group by month
formated_gdf_copy = pd.DataFrame(formated_gdf)
formated_gdf_copy = pd.merge(formated_gdf_copy,formated_gdf_copy['date'].str.split('/', expand=True),left_index=True, right_index=True)
formated_gdf_copy.rename(columns = {0:"month", 1:"day",2:"year"}, inplace = True)
formated_gdf_copy["date"] = formated_gdf_copy["year"] + "/" +formated_gdf_copy["month"]
formated_gdf_copy["name"] = formated_gdf["name"]
formated_gdf_copy = formated_gdf_copy.groupby( ['date','id','name']).mean().reset_index()
formated_gdf_copy.rename(columns = {0:"month", 1:"day",2:"year",
                                    "retail_and_recreation_percent_change_from_baseline":"Mobility: retail and recreation"}, inplace = True)

In [ ]:
# Dynamic cartogram: mobility in the areas of commerce and entertainment in the provinces
# in the following months of the pandemic (opens in a separate window after a while)

# Cartogram
fig = px.choropleth(
    formated_gdf_copy,
    locations="id",
    geojson=poland_states,
    color="Mobility: retail and recreation",
    hover_name="name",
    hover_data=["Mobility: retail and recreation"],
    animation_frame='date',
    title="Mobility: retail and recreation",
)
fig.update_geos(fitbounds="locations", visible=False)

#fig.show()
#fig.write_html("cartogram.html")